In [14]:

### File Address: https://www.sample-videos.com/
%load_ext autoreload
%autoreload 2
import json
import time
import os
from blake3 import blake3
import warnings
warnings.filterwarnings("ignore")
video_path = "demo-3s-small.mp4"
input_height = 240
input_width = 320
output_height = 120
output_width = 160
current_dir = os.getcwd()
base_dir = os.path.dirname(current_dir)
snark_path = "/Users/jianfeng/Developer/fibonacci/video-resizing-freivalds"
target_pk_file = os.path.join(base_dir, "demo", "test_pk.bin")
target_vk_file = os.path.join(base_dir, "demo", "test_vk.bin")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Init ELF

In [27]:
start_time = time.time()
os.system(f"cd {snark_path}/program-precompile && RUSTFLAGS='-Awarnings' cargo prove build")
end_time = time.time()
print(f"Time taken to build ELF: {end_time - start_time} seconds")

cargo:warning=rustc +succinct --version: "rustc 1.82.0-dev\n"


[sp1]     Compiling freivalds-precompile v0.1.0 (/Users/jianfeng/Developer/fibonacci/video-resizing-freivalds/program-precompile)
[sp1]  warning: unused import: `blake3::hash`
[sp1]   --> program-precompile/src/main.rs:3:5
[sp1]    |
[sp1]  3 | use blake3::hash;
[sp1]    |     ^^^^^^^^^^^^
[sp1]    |
[sp1]    = note: `#[warn(unused_imports)]` on by default
[sp1]  


cargo:rustc-env=SP1_ELF_freivalds-precompile=/Users/jianfeng/Developer/fibonacci/video-resizing-freivalds/target/elf-compilation/riscv32im-succinct-zkvm-elf/release/freivalds-precompile
Time taken to build ELF: 2.194211006164551 seconds


[sp1]  warning: `freivalds-precompile` (bin "freivalds-precompile") generated 1 warning (run `cargo fix --bin "freivalds-precompile"` to apply 1 suggestion)
[sp1]      Finished `release` profile [optimized] target(s) in 0.80s


# Verifier Setup Pk and Vk

In [28]:
start_time = time.time()
config = {
    "target_pk_file": target_pk_file,
    "target_vk_file": target_vk_file
}
with open(os.path.join(snark_path, "init", "config.json"), "w") as f:
    json.dump(config, f)
os.system(f"cd {snark_path}/init && RUST_LOG=info cargo run --release -- --prove")
end_time = time.time()
print(f"Time taken to generate Pk and Vk: {end_time - start_time} seconds")

   Compiling init v0.1.0 (/Users/jianfeng/Developer/fibonacci/video-resizing-freivalds/init)
 --> init/src/main.rs:1:36
  |
1 | use sp1_sdk::{utils, ProverClient, SP1ProofWithPublicValues, SP1Stdin, SP1VerifyingKey};
  |                                    ^^^^^^^^^^^^^^^^^^^^^^^^  ^^^^^^^^  ^^^^^^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> init/src/main.rs:2:5
  |
2 | use std::io::Write;
  |     ^^^^^^^^^^^^^^

 --> init/src/main.rs:4:5
  |
4 | use clap::Parser;
  |     ^^^^^^^^^^^^

  --> init/src/main.rs:13:9
   |
13 |     let client = ProverClient::from_env();
   |         ^^^^^^ help: if this is intentional, prefix it with an underscore: `_client`
   |
   = note: `#[warn(unused_variables)]` on by default

    Finished `release` profile [optimized] target(s) in 3.77s
     Running `/Users/jianfeng/Developer/fibonacci/video-resizing-freivalds/target/release/init --prove`


2025-05-16T00:39:50.945416Z  WARN SP1_PROVER environment variable not set, defaulting to 'cpu'    
2025-05-16T00:39:51.639481Z  INFO vk verification: true
2025-05-16T00:39:54.466696Z  WARN SP1_PROVER environment variable not set, defaulting to 'cpu'    
2025-05-16T00:39:55.119424Z  INFO vk verification: true
Successfully Save Pk and Vk
Time taken to generate Pk and Vk: 12.901772022247314 seconds


# Prover

### Decompose images from the video

In [29]:
# Use ffmpeg to decompose
import os
# Create directories if they don't exist
if not os.path.exists("frames"):
    os.makedirs("frames")
if not os.path.exists("decomposed_frames"):
    os.makedirs("decomposed_frames")

os.system(f"ffmpeg -i {video_path} frames/output_%04d.png")

ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --enable-debug=3 --disable-optimizations --disable-stripping
  libavutil      59.  8.100 / 59.  8.100
  libavcodec     61.  3.100 / 61.  3.100
  libavformat    61.  1.100 / 61.  1.100
  libavdevice    61.  1.100 / 61.  1.100
  libavfilter    10.  1.100 / 10.  1.100
  libswscale      8.  1.100 /  8.  1.100
  libswresample   5.  1.100 /  5.  1.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'demo-3s-small.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf61.7.100
  Duration: 00:00:03.08, start: 0.000000, bitrate: 775 kb/s
  Stream #0:0[0x1](und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, 5.1, fltp, 348 kb/s (default)
      Metadata:
        handler_name    : SoundHandler
        vendor_id       : [0][0][0][0]
  Stream #0:1[0x2](und): Video: h264 (High) (avc1 / 

0

In [30]:
import sys
sys.path.append(os.path.abspath("../"))
from scripts.decompose_image import decompose_image 
for i in sorted(os.listdir("frames")):
    if i.endswith(".png"):
        r,g,b = decompose_image(os.path.join("frames", i), os.path.join("decomposed_frames", i[:-4]))

Saved R channel to decomposed_frames/output_0001_R.txt
Saved G channel to decomposed_frames/output_0001_G.txt
Saved B channel to decomposed_frames/output_0001_B.txt
Saved R channel to decomposed_frames/output_0002_R.txt
Saved G channel to decomposed_frames/output_0002_G.txt
Saved B channel to decomposed_frames/output_0002_B.txt
Saved R channel to decomposed_frames/output_0003_R.txt
Saved G channel to decomposed_frames/output_0003_G.txt
Saved B channel to decomposed_frames/output_0003_B.txt
Saved R channel to decomposed_frames/output_0004_R.txt
Saved G channel to decomposed_frames/output_0004_G.txt
Saved B channel to decomposed_frames/output_0004_B.txt
Saved R channel to decomposed_frames/output_0005_R.txt
Saved G channel to decomposed_frames/output_0005_G.txt
Saved B channel to decomposed_frames/output_0005_B.txt
Saved R channel to decomposed_frames/output_0006_R.txt
Saved G channel to decomposed_frames/output_0006_G.txt
Saved B channel to decomposed_frames/output_0006_B.txt
Saved R ch

In [31]:
### Generate the targeted File 
## Shared knowledge between prover and verifier
if os.path.exists("resized.mp4"):
    os.remove("resized.mp4")
os.system(f"ffmpeg -i {video_path} -vf scale=160:120 resized.mp4")
if not os.path.exists("resized_frames"):
    os.makedirs("resized_frames")
if not os.path.exists("decomposed_resized_frames"):
    os.makedirs("decomposed_resized_frames")

# Check if resized.mp4 exists and ask for overwrite permission

os.system(f"ffmpeg -i resized.mp4 resized_frames/output_%04d.png")
for i in sorted(os.listdir("frames")):
    if i.endswith(".png"):
        r,g,b = decompose_image(os.path.join("resized_frames", i), os.path.join("decomposed_resized_frames", i[:-4]))

ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --enable-debug=3 --disable-optimizations --disable-stripping
  libavutil      59.  8.100 / 59.  8.100
  libavcodec     61.  3.100 / 61.  3.100
  libavformat    61.  1.100 / 61.  1.100
  libavdevice    61.  1.100 / 61.  1.100
  libavfilter    10.  1.100 / 10.  1.100
  libswscale      8.  1.100 /  8.  1.100
  libswresample   5.  1.100 /  5.  1.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'demo-3s-small.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf61.7.100
  Duration: 00:00:03.08, start: 0.000000, bitrate: 775 kb/s
  Stream #0:0[0x1](und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, 5.1, fltp, 348 kb/s (default)
      Metadata:
        handler_name    : SoundHandler
        vendor_id       : [0][0][0][0]
  Stream #0:1[0x2](und): Video: h264 (High) (avc1 / 

======================= sws_init_swscale ========================
c->srcBpc: 8
c->dstBpc: 8
c->srcFormat: yuv420p
c->dstFormat: yuv420p
c->srcW: 320, c->srcH: 240
c->dstW: 160, c->dstH: 120
c->flags: 4
======================= ff_sws_init_swscale_aarch64 ========================
have_neon(cpu_flags) = 1
============== ff_init_filters ==============
============= num_vdesc = 2
============= need_lum_conv = 0
============= need_chr_conv = 0
============= need_gamma = 0
============= dst_stride = 400
============= c->numSlice = 3
============= c->numDesc = 4
============= c->descIndex[0] = 1
============= c->descIndex[1] = 2
Pixel format: AV_PIX_FMT_YUV420P
============= c->srcH = 240
============= c->chrSrcH = 120
============= c->chrSrcHSubSample = 1
============= c->chrSrcVSubSample = 1
Pixel format for horizontal scaler output : AV_PIX_FMT_YUV420P
============= lumBufSize = 14
============= chrBufSize = 12
============= c->chrDstHSubSample = 1
============= c->chrDstVSubSample = 1
Pixe

[out#0/mp4 @ 0x153e280d0] video:110KiB audio:128KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 1.135036%
frame=   47 fps=0.0 q=2.0 Lsize=     241KiB time=00:00:03.09 bitrate= 637.9kbits/s speed=8.06x    
[aac @ 0x143e30ac0] Qavg: 296.967
ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --enable-debug=3 --disable-optimizations --disable-stripping
  libavutil      59.  8.100 / 59.  8.100
  libavcodec     61.  3.100 / 61.  3.100
  libavformat    61.  1.100 / 61.  1.100
  libavdevice    61.  1.100 / 61.  1.100
  libavfilter    10.  1.100 / 10.  1.100
  libswscale      8.  1.100 /  8.  1.100
  libswresample   5.  1.100 /  5.  1.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'resized.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2mp41
    encoder         : Lavf61.1.100
  Duration: 00:00:03.13, start: 0.000000, bitrate: 62

Saved R channel to decomposed_resized_frames/output_0005_R.txt
Saved G channel to decomposed_resized_frames/output_0005_G.txt
Saved B channel to decomposed_resized_frames/output_0005_B.txt
Saved R channel to decomposed_resized_frames/output_0006_R.txt
Saved G channel to decomposed_resized_frames/output_0006_G.txt
Saved B channel to decomposed_resized_frames/output_0006_B.txt
Saved R channel to decomposed_resized_frames/output_0007_R.txt
Saved G channel to decomposed_resized_frames/output_0007_G.txt
Saved B channel to decomposed_resized_frames/output_0007_B.txt
Saved R channel to decomposed_resized_frames/output_0008_R.txt
Saved G channel to decomposed_resized_frames/output_0008_G.txt
Saved B channel to decomposed_resized_frames/output_0008_B.txt
Saved R channel to decomposed_resized_frames/output_0009_R.txt
Saved G channel to decomposed_resized_frames/output_0009_G.txt
Saved B channel to decomposed_resized_frames/output_0009_B.txt
Saved R channel to decomposed_resized_frames/output_001

In [32]:
# Generate Proof from rust 
if not os.path.exists("proof"):
    os.makedirs("proof")
current_dir = os.getcwd()
base_dir = os.path.dirname(current_dir)
test_counts = 5
for i in sorted(os.listdir("decomposed_frames")):
    start_time = time.time()
    print("generating proof for ", i)
    if i.endswith(".txt"):
        config = {
            "input_width": input_width,
            "input_height": input_height,
            "output_width": output_width,
            "output_height": output_height,
            "input_file": os.path.join(base_dir, "demo/decomposed_frames", i),
            "target_file": os.path.join(base_dir, "demo/decomposed_resized_frames", i),
            "target_prove_file": os.path.join(base_dir, "demo/proof", i[:-4] + "_proof-with-pis.bin"),
            "target_pk_file": target_pk_file
        }
        with open(os.path.join(snark_path, "script", "config.json"), "w") as f:
            json.dump(config, f)
        os.system(f"cd {snark_path}/script && RUST_LOG=info cargo run --release -- --prove")
    end_time = time.time()
    print(f"Time taken to generate proof for {i}: {end_time - start_time} seconds")
    test_counts -= 1
    if test_counts == 0:
        break    


generating proof for  output_0001_B.txt


   Compiling script v0.1.0 (/Users/jianfeng/Developer/fibonacci/video-resizing-freivalds/script)
 --> script/src/bin/main.rs:1:11
  |
1 | use lib::{generate_horizontal_filter, generate_vertical_filter, load_image_from_file};
  |           ^^^^^^^^^^^^^^^^^^^^^^^^^^  ^^^^^^^^^^^^^^^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> script/src/bin/main.rs:2:72
  |
2 | use sp1_sdk::{utils, ProverClient, SP1ProofWithPublicValues, SP1Stdin, SP1VerifyingKey};
  |                                                                        ^^^^^^^^^^^^^^^

 --> script/src/bin/main.rs:5:5
  |
5 | use clap::Parser;
  |     ^^^^^^^^^^^^

 --> script/src/bin/main.rs:7:14
  |
7 | use blake3::{hash};
  |              ^^^^

   --> script/src/bin/main.rs:118:27
    |
118 |             let product = (freivalds_left[i] as u32 * h_matrix[i][j] as u32);
    |                           ^                                                ^
    |
    = note: `#[warn(unused_parens)]` on by default


input_path: "/Users/jianfeng/Developer/fibonacci/demo/decomposed_frames/output_0001_B.txt"
input_path: "/Users/jianfeng/Developer/fibonacci/demo/decomposed_resized_frames/output_0001_B.txt"
2025-05-16T00:41:00.907931Z  WARN SP1_PROVER environment variable not set, defaulting to 'cpu'    
2025-05-16T00:41:01.617451Z  INFO vk verification: true
2025-05-16T00:41:04.951856Z  INFO prove_core: clk = 0 pc = 0x20209c    
2025-05-16T00:41:04.951991Z  INFO prove_core: ┌╴setup
2025-05-16T00:41:04.965882Z  INFO prove_core: clk = 0 pc = 0x20209c    
2025-05-16T00:41:04.965972Z  INFO prove_core: ┌╴setup
2025-05-16T00:41:04.973053Z  WARN prove_core: stopping shard early due to no shapes fitting: clk: 19,
                            clk_usage: 18.884110882664185    
2025-05-16T00:41:04.983886Z  WARN prove_core: stopping shard early due to no shapes fitting: clk: 19,
                            clk_usage: 18.884838278917112    
2025-05-16T00:41:05.002620Z  INFO prove_core: deferred 0 records    
2025-0

stderr: cycle-tracker-start: main
stderr: cycle-tracker-start: main
stderr: cycle-tracker-end: main


2025-05-16T00:41:05.499007Z  INFO prove_core:generate main traces: close time.busy=442ms time.idle=1.50µs index=0
2025-05-16T00:41:05.589274Z  WARN prove_core: stopping shard early due to no shapes fitting: clk: 19,
                            clk_usage: 18.884110882664185    
2025-05-16T00:41:05.589427Z  INFO prove_core: deferred 0 records    
2025-05-16T00:41:05.671259Z  INFO prove_core: Shard Lifted: Index=2, Cluster=195
2025-05-16T00:41:05.671272Z  INFO prove_core: Chip Cpu: 19  -> 19 
2025-05-16T00:41:05.671277Z  INFO prove_core: Chip AddSub: 19  -> 19 
2025-05-16T00:41:05.671278Z  INFO prove_core: Chip Bitwise: 0   -> 15 
2025-05-16T00:41:05.671280Z  INFO prove_core: Chip Mul: 16  -> 16 
2025-05-16T00:41:05.671281Z  INFO prove_core: Chip ShiftRight: 0   -> 15 
2025-05-16T00:41:05.671282Z  INFO prove_core: Chip ShiftLeft: 7   -> 15 
2025-05-16T00:41:05.671283Z  INFO prove_core: Chip Lt: 18  -> 18 
2025-05-16T00:41:05.671284Z  INFO prove_core: Chip MemoryLocal: 12  -> 16 
2025-05-1

...

input_path: "/Users/jianfeng/Developer/fibonacci/demo/decomposed_frames/output_0002_G.txt"
input_path: "/Users/jianfeng/Developer/fibonacci/demo/decomposed_resized_frames/output_0002_G.txt"
2025-05-16T00:48:23.272206Z  WARN SP1_PROVER environment variable not set, defaulting to 'cpu'    
2025-05-16T00:48:23.941418Z  INFO vk verification: true
2025-05-16T00:48:27.089378Z  INFO prove_core: clk = 0 pc = 0x20209c    
2025-05-16T00:48:27.089517Z  INFO prove_core: ┌╴setup
2025-05-16T00:48:27.103184Z  INFO prove_core: clk = 0 pc = 0x20209c    
2025-05-16T00:48:27.103257Z  INFO prove_core: ┌╴setup
2025-05-16T00:48:27.110503Z  WARN prove_core: stopping shard early due to no shapes fitting: clk: 19,
                            clk_usage: 18.884110882664185    
2025-05-16T00:48:27.120952Z  WARN prove_core: stopping shard early due to no shapes fitting: clk: 19,
                            clk_usage: 18.884838278917112    
2025-05-16T00:48:27.138485Z  INFO prove_core: deferred 0 records    
2025-0

stderr: cycle-tracker-start: main
stderr: cycle-tracker-start: main
stderr: cycle-tracker-end: main


2025-05-16T00:48:27.552788Z  INFO prove_core:generate main traces: close time.busy=358ms time.idle=1.58µs index=0
2025-05-16T00:48:27.591484Z  WARN prove_core: stopping shard early due to no shapes fitting: clk: 19,
                            clk_usage: 18.884110882664185    
2025-05-16T00:48:27.591638Z  INFO prove_core: deferred 0 records    
2025-05-16T00:48:27.670802Z  INFO prove_core: Shard Lifted: Index=2, Cluster=195
2025-05-16T00:48:27.670812Z  INFO prove_core: Chip Cpu: 19  -> 19 
2025-05-16T00:48:27.670814Z  INFO prove_core: Chip AddSub: 19  -> 19 
2025-05-16T00:48:27.670815Z  INFO prove_core: Chip Bitwise: 0   -> 15 
2025-05-16T00:48:27.670816Z  INFO prove_core: Chip Mul: 16  -> 16 
2025-05-16T00:48:27.670817Z  INFO prove_core: Chip ShiftRight: 0   -> 15 
2025-05-16T00:48:27.670818Z  INFO prove_core: Chip ShiftLeft: 7   -> 15 
2025-05-16T00:48:27.670819Z  INFO prove_core: Chip Lt: 18  -> 18 
2025-05-16T00:48:27.670821Z  INFO prove_core: Chip MemoryLocal: 12  -> 16 
2025-05-1

stderr: cycle-tracker-end: main


2025-05-16T00:49:11.858813Z  INFO prove_core:generate main traces: close time.busy=899ms time.idle=2.25µs index=4
2025-05-16T00:50:18.745367Z  INFO prove_core: execution report (totals): total_cycles=2346490, total_syscall_cycles=71, touched_memory_addresses=87615
2025-05-16T00:50:18.747594Z  INFO prove_core: execution report (opcode counts):
2025-05-16T00:50:18.748170Z  INFO prove_core:   765039 add
2025-05-16T00:50:18.748172Z  INFO prove_core:   295504 lw
2025-05-16T00:50:18.748174Z  INFO prove_core:   197146 lbu
2025-05-16T00:50:18.748175Z  INFO prove_core:   193067 sw
2025-05-16T00:50:18.748176Z  INFO prove_core:   147713 sll
2025-05-16T00:50:18.748177Z  INFO prove_core:   147492 or
2025-05-16T00:50:18.748178Z  INFO prove_core:   116933 bne
2025-05-16T00:50:18.748180Z  INFO prove_core:   115559 bgeu
2025-05-16T00:50:18.748181Z  INFO prove_core:    96393 mul
2025-05-16T00:50:18.748182Z  INFO prove_core:    92162 srl
2025-05-16T00:50:18.748183Z  INFO prove_core:    74533 xor
2025-05-

# Verifier

In [23]:
if not os.path.exists("resized_frames"):
    os.makedirs("resized_frames")
if not os.path.exists("decomposed_resized_frames"):
    os.makedirs("decomposed_resized_frames")

# Check if resized.mp4 exists and ask for overwrite permission

os.system(f"ffmpeg -i resized.mp4 resized_frames/output_%04d.png")
for i in sorted(os.listdir("frames")):
    if i.endswith(".png"):
        r,g,b = decompose_image(os.path.join("resized_frames", i), os.path.join("decomposed_resized_frames", i[:-4]))

ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --enable-debug=3 --disable-optimizations --disable-stripping
  libavutil      59.  8.100 / 59.  8.100
  libavcodec     61.  3.100 / 61.  3.100
  libavformat    61.  1.100 / 61.  1.100
  libavdevice    61.  1.100 / 61.  1.100
  libavfilter    10.  1.100 / 10.  1.100
  libswscale      8.  1.100 /  8.  1.100
  libswresample   5.  1.100 /  5.  1.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'resized.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2mp41
    encoder         : Lavf61.1.100
  Duration: 00:00:03.13, start: 0.000000, bitrate: 629 kb/s
  Stream #0:0[0x1](und): Video: mpeg4 (Simple Profile) (mp4v / 0x7634706D), yuv420p, 160x120 [SAR 1:1 DAR 4:3], 287 kb/s, 15 fps, 15 tbr, 15360 tbn (default)
      Metadata:
        handler_name    : VideoHandler
        vendor_id       : [0][0][0][0]
      

Saved R channel to decomposed_resized_frames/output_0001_R.txt
Saved G channel to decomposed_resized_frames/output_0001_G.txt
Saved B channel to decomposed_resized_frames/output_0001_B.txt
Saved R channel to decomposed_resized_frames/output_0002_R.txt
Saved G channel to decomposed_resized_frames/output_0002_G.txt
Saved B channel to decomposed_resized_frames/output_0002_B.txt
Saved R channel to decomposed_resized_frames/output_0003_R.txt
Saved G channel to decomposed_resized_frames/output_0003_G.txt
Saved B channel to decomposed_resized_frames/output_0003_B.txt
Saved R channel to decomposed_resized_frames/output_0004_R.txt
Saved G channel to decomposed_resized_frames/output_0004_G.txt
Saved B channel to decomposed_resized_frames/output_0004_B.txt
Saved R channel to decomposed_resized_frames/output_0005_R.txt
Saved G channel to decomposed_resized_frames/output_0005_G.txt
Saved B channel to decomposed_resized_frames/output_0005_B.txt
Saved R channel to decomposed_resized_frames/output_000

In [24]:
test_counts = 3
for i in sorted(os.listdir("decomposed_resized_frames")):
    print("verifying proof for ", i)
    if i.endswith(".txt"):
        start_time = time.time()
        prove_file = os.path.join(base_dir, "demo/proof", i[:-4] + "_proof-with-pis.bin")
        config = {
            "target_prove_file": prove_file,
            "target_vk_file": target_vk_file
        }
        with open(os.path.join(snark_path, "verifier", "config.json"), "w") as f:
            json.dump(config, f)
        os.system(f"cd {snark_path}/verifier && cargo run")
        data = []
        with open(os.path.join(base_dir, "demo/decomposed_resized_frames", i), "r") as f:
            for line in f:
                # Split line by spaces and convert each number
                numbers = [int(x) for x in line.strip().split()]
                data.extend(numbers)
        hash_obj = blake3(bytes(data))
        print(hash_obj.hexdigest())
        end_time = time.time()
        print(f"Time taken to verify proof for {i}: {end_time - start_time} seconds")
    test_counts -= 1
    if test_counts == 0:
        break

verifying proof for  output_0001_B.txt


   Compiling proc-macro2 v1.0.93
   Compiling unicode-ident v1.0.13
   Compiling cfg-if v1.0.0
   Compiling libc v0.2.158
   Compiling autocfg v1.3.0
   Compiling once_cell v1.19.0
   Compiling serde v1.0.210
   Compiling version_check v0.9.5
   Compiling pin-project-lite v0.2.14
   Compiling byteorder v1.5.0
   Compiling either v1.13.0
   Compiling allocator-api2 v0.2.18
   Compiling syn v1.0.109
   Compiling itoa v1.0.11
   Compiling num-traits v0.2.19
   Compiling equivalent v1.0.1
   Compiling subtle v2.6.1
   Compiling crossbeam-utils v0.8.20
   Compiling memchr v2.7.4
   Compiling ahash v0.8.11
   Compiling rayon-core v1.12.1
   Compiling log v0.4.22
   Compiling typenum v1.17.0
   Compiling itertools v0.12.1
   Compiling quote v1.0.37
   Compiling getrandom v0.2.15
   Compiling syn v2.0.98
   Compiling rand_core v0.6.4
   Compiling num-integer v0.1.46
   Compiling tracing-core v0.1.32
   Compiling generic-array v0.14.7
   Compiling libm v0.2.8
   Compiling crossbeam-epoch v0.9.1

equal_sum: true
exceed_limit_20: 664
exceed_limit_50: 30
hash_target_image: Hash("8272c2667527755d033ba06786d68b118d2c6deaac1a10b144d09afe2e5c9b02")
8272c2667527755d033ba06786d68b118d2c6deaac1a10b144d09afe2e5c9b02
Time taken to verify proof for output_0001_B.txt: 127.45935988426208 seconds
verifying proof for  output_0001_G.txt


 --> verifier/src/main.rs:1:62
  |
1 | use sp1_sdk::{utils, ProverClient, SP1ProofWithPublicValues, SP1Stdin, SP1VerifyingKey};
  |                                                              ^^^^^^^^  ^^^^^^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> verifier/src/main.rs:2:5
  |
2 | use std::io::Write;
  |     ^^^^^^^^^^^^^^

 --> verifier/src/main.rs:3:5
  |
3 | use blake3::hash;
  |     ^^^^^^^^^^^^

    Finished `dev` profile [unoptimized + debuginfo] target(s) in 1.48s
     Running `/Users/jianfeng/Developer/fibonacci/video-resizing-freivalds/target/debug/verifier`


equal_sum: true
exceed_limit_20: 118
exceed_limit_50: 0
hash_target_image: Hash("28d7fb4428ef7bbde4f44e19c04c79d3011d57d52a886cbd068817d9db84bdd9")
28d7fb4428ef7bbde4f44e19c04c79d3011d57d52a886cbd068817d9db84bdd9
Time taken to verify proof for output_0001_G.txt: 43.40147089958191 seconds
verifying proof for  output_0001_R.txt


 --> verifier/src/main.rs:1:62
  |
1 | use sp1_sdk::{utils, ProverClient, SP1ProofWithPublicValues, SP1Stdin, SP1VerifyingKey};
  |                                                              ^^^^^^^^  ^^^^^^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> verifier/src/main.rs:2:5
  |
2 | use std::io::Write;
  |     ^^^^^^^^^^^^^^

 --> verifier/src/main.rs:3:5
  |
3 | use blake3::hash;
  |     ^^^^^^^^^^^^

    Finished `dev` profile [unoptimized + debuginfo] target(s) in 0.61s
     Running `/Users/jianfeng/Developer/fibonacci/video-resizing-freivalds/target/debug/verifier`


equal_sum: true
exceed_limit_20: 158
exceed_limit_50: 0
hash_target_image: Hash("ba429e021a80ebc100838fd0457fc05edc22d494a3f9e656eb2ae27463d659f5")
ba429e021a80ebc100838fd0457fc05edc22d494a3f9e656eb2ae27463d659f5
Time taken to verify proof for output_0001_R.txt: 41.65486288070679 seconds
